# Final project: Activity Recognition

The final assignment is to implement a classifier for activity recognition based on data from the phone’s sensors.

For this task, first the acceleration using the sensors of a phone was recorded. Then an HMM classifier that should be able to discriminate between, standing still, walking and running was implemented and its functionality verified.

In [32]:
%reload_ext autoreload
%autoreload 2

from PattRecClasses import DiscreteD, GaussD, HMM, MarkovChain
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

### Data - Import and Pre-processing

In [40]:
# import
train = pd.read_csv('Data/data.csv')
test_stand =  pd.read_csv('Data/Stand3.csv')
test_walk =  pd.read_csv('Data/Run3.csv')
test_run =  pd.read_csv('Data/Walk3.csv')
# drop unnecessary columns
train = train.drop(columns = ['time', 'atotal'])
test_stand = test_stand.drop(columns = ['time', 'atotal'])
test_walk = test_walk.drop(columns = ['time', 'atotal'])
test_run = test_run.drop(columns = ['time', 'atotal'])
# moving average + drop NaN due to filtering
train = train.rolling(window=10).mean().dropna()
test_stand = test_stand.rolling(window=10).mean().dropna()
test_walk = test_walk.rolling(window=10).mean().dropna()
test_run = test_run.rolling(window=10).mean().dropna()
# convert dataframe into array
train = train.values
test_stand = test_stand.values
test_walk = test_walk.values
test_run = test_run.values

mean_stand = np.mean(test_stand, axis = 0)
centered_data = test_stand - mean_stand
cov_stand = np.dot(centered_data.T, centered_data) / (len(test_stand) - 1)
mean_walk = np.mean(test_walk, axis = 0)
centered_data = test_walk - mean_walk
cov_walk = np.dot(centered_data.T, centered_data) / (len(test_walk) - 1)
mean_run = np.mean(test_run, axis = 0)
centered_data = test_run - mean_run
cov_run = np.dot(centered_data.T, centered_data) / (len(test_run) - 1)


### Create and initialize HMM

In [44]:
# infinite HMM, with three states: standing (S1), walking (S2), running (S3)

q = np.array([1/3, 1/3, 1/3])
A  = np.array([[0.9, 0.05, 0.05], [0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])
mc = MarkovChain(q, A) 

g1 = GaussD(means=mean_stand, cov = cov_stand) # Distribution for state 'standing'
g2 = GaussD(means=mean_walk, cov = cov_walk) # Distribution for state 'walking'
g3 = GaussD(means=mean_run, cov = cov_run) # Distribution for state 'running'
h  = HMM(mc, [g1, g2, g3])           # The HMM


h.train(train)



In [45]:
h.stateGen.q = np.array([1/3, 1/3, 1/3])

### Train HMM

In [50]:
states_stand = h.viterbi(test_stand)
states_walk = h.viterbi(test_walk)
states_run = h.viterbi(test_run)

states_stand

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

### Verify HMM